# Learning Objectives

In this notebook, you will 
- learn the concept of ETL
- write ETL jobs for CSV files from `pgexercises` https://pgexercises.com/gettingstarted.html

# What's ETL or ELT?

ETL stands for Extract, Transform, Load. In the context of Spark, ETL refers to the process of extracting data from various sources, transforming it into a desired format or structure, and loading it into a target system, such as a data warehouse or a data lake.

Here's a breakdown of each step in the ETL process:

## Extract
This step involves extracting data from multiple sources, such as databases, files (CSV, JSON, Parquet), APIs, or streaming data sources. Spark provides connectors and APIs to read data from a wide range of sources, allowing you to extract data in parallel and efficiently handle large datasets.

## Transform
In the transform step, the extracted data is processed and transformed according to specific business logic or requirements. This may involve cleaning the data, applying calculations or aggregations, performing data enrichment, filtering, joining datasets, or any other data manipulation operations. Spark provides a powerful set of transformation functions and SQL capabilities to perform these operations efficiently in a distributed and scalable manner.

## Load
Once the data has been transformed, it is loaded into a target system, such as a data warehouse, a data lake, or another storage system. Spark allows you to write the transformed data to various output formats and storage systems, including databases, distributed file systems (like Hadoop Distributed File System or Amazon S3), or columnar formats like Delta Lake or Apache Parquet. The data can be partitioned, sorted, or structured to optimize querying and analysis.

Spark's distributed computing capabilities, scalability, and rich ecosystem of libraries make it a popular choice for ETL workflows. It can handle large-scale data processing, perform complex transformations, and efficiently load data into different target systems.

By leveraging Spark for ETL, organizations can extract data from diverse sources, apply transformations to ensure data quality and consistency, and load the transformed data into a central repository for further analysis, reporting, or machine learning tasks.

## Import `pgexercises` CSV files

- The pgexercises CSV data files can be found [here](https://github.com/jarviscanada/jarvis_data_eng_demo/tree/feature/data/spark/data/pgexercises).
- The pgexercises schema can be found [here](https://pgexercises.com/gettingstarted.html) (for reference purposes).
- Upload the `bookings.csv`, `facilities.csv`, and `members.csv` files using Databricks UI (see screenshot)
- You can view the imported files from the DBFS UI.

![Upload Files](https://raw.githubusercontent.com/jarviscanada/jarvis_data_eng_demo/feature/data/spark/notebook/spark_fundamentals/img/upload%20file.png)

# Interview Questions

While completing the rest of the practice, try to answer the following questions:

## Concepts
- What is ETL? (Hint: Explain each step)

## Databricks
- What is Databricks?
- What is a Notebook?
- What is DBFS?
- What is a cluster? 
- Is Databricks a data lake or a data warehouse?

## Managed Table
- What is a managed table in Databricks?
- Can you explain how to create a managed table in Databricks?
- Can you compare a managed table with an RDBMS table? (Hint: Schema on read vs schema on write)
- What is the Hive metastore and how does it relate to managed tables in Databricks?
- How does a managed table differ from an unmanaged (external) table in Databricks? (Hint: Consider what happens to the data when the table is deleted)
- How can you define a schema for a managed table?

## Spark
`df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_location)`
- What does the option("inferSchema", "true") do? 
- What does the option("header", "true") do?
- How can you write data to a managed table?
- How can you read data from a managed table into a DataFrame?

# ETL `bookings.csv` file

- **Extract**: Load data from CSV file into a DF
- **Transformation**: no transformation needed as we want to load data as it
- **Load**: Save the DF into a managed table (or Hive table); 

# Managed Table
This is an important interview topic. Some people may refer to managed tables as Hive tables.

https://docs.databricks.com/data-governance/unity-catalog/create-tables.html

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType

file_location = "/FileStore/tables/bookings.csv"

# What does `option("header", "true")` and `option("inferSchema", "true")` do?
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_location)

# Why the df schema doesn't match the DDL data type? https://pgexercises.com/gettingstarted.html (hint: `option("inferSchema", "true")`)
df.printSchema()

# Here is the solution to define schema manually
# Define schema for the bookings table
schema = StructType([
    StructField("bookid", IntegerType(), nullable = False),
    StructField("facid", IntegerType(), nullable = False),
    StructField("memid", IntegerType(), nullable = False),
    StructField("starttime", TimestampType(), nullable = False),
    StructField("slots", IntegerType(), nullable = False)
])

# Read data from CSV file into DataFrame with predefined schema
df = spark.read.format("csv").option("header", "true").schema(schema).load(file_location)

# No 

# Drop the table if it already exists
spark.sql("DROP TABLE IF EXISTS bookings")

# Write data from DataFrame into managed table
df.write.saveAsTable("bookings")


root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)



# Complete ETL Jobs

- Complete ETL for `facilities.csv` and `members.csv`
- Tips
  - The Databricks community version will terminate the cluster after a few hours of inactivity. As a result, all managed tables will be deleted. You will need to rerun this notebook to perform the ETL on all files for the other exercises.
  - DBFS data will not be deleted when a custer become inactive/deleted

In [0]:
# Write a ETL job for `facilities.csv`
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, IntegerType

file_location_fac = '/FileStore/tables/facilities.csv'

schema_fac = StructType([
    StructField("facid", IntegerType(), nullable = False),
    StructField("name", StringType(), nullable = False),
    StructField("membercost", DecimalType(), nullable = False),
    StructField("guestcost", DecimalType(), nullable = False),
    StructField("initialoutlay", DecimalType(), nullable = False),
    StructField("monthlymaintenance", DecimalType(), nullable = False)
])

df_fac = spark.read.format(
    "csv"
).option(
    "header",
    "true"
).schema(schema_fac).load(file_location_fac)

spark.sql("DROP TABLE IF EXISTS facilities")

df_fac.write.saveAsTable("facilities")

In [0]:
# Write a ETL job Complete ETL for `members.csv`
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, IntegerType

file_location_mem = '/FileStore/tables/members.csv'

schema_mem = StructType([
    StructField("memid", IntegerType(), nullable = False),
    StructField("surname", StringType(), nullable = False),
    StructField("firstname", StringType(), nullable = False),
    StructField("address", StringType(), nullable = False),
    StructField("zipcode", IntegerType(), nullable = False),
    StructField("telephone", StringType(), nullable = False),
    StructField("recommendedby", IntegerType(), nullable = True),
    StructField("joindate", TimestampType(), nullable = False)
])

df_mem = spark.read.format(
    "csv"
).option(
    "header",
    "true"
).schema(schema_mem).load(file_location_mem)

spark.sql("DROP TABLE IF EXISTS members")

df_mem.write.saveAsTable("members")

In [0]:
# Verify
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Verify Table Content").getOrCreate()
query = f"SELECT * FROM members LIMIT 5"
result = spark.sql(query)
result.show()

+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|memid| surname|firstname|             address|zipcode|     telephone|recommendedby|           joindate|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|    0|   GUEST|    GUEST|               GUEST|      0|(000) 000-0000|         NULL|2012-07-01 00:00:00|
|    1|   Smith|   Darren|8 Bloomsbury Clos...|   4321|  555-555-5555|         NULL|2012-07-02 12:02:05|
|    2|   Smith|    Tracy|8 Bloomsbury Clos...|   4321|  555-555-5555|         NULL|2012-07-02 12:08:23|
|    3|  Rownam|      Tim|23 Highway Way, B...|  23423|(844) 693-0723|         NULL|2012-07-03 09:32:15|
|    4|Joplette|   Janice|20 Crossing Road,...|    234|(833) 942-4710|            1|2012-07-03 10:25:05|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+



# Save your work to Git

- Export the notebook to IPYTHON format, `notebook top menu bar -> File -> Export -> iphython`
- Upload to your Git repository, `your_repo/spark/notebooks/`
- Github can render ipython notebook https://github.com/josephcslater/JupyterExamples/blob/master/Calc_Review.ipynb